In [1]:
import pandas as pd

colunas = ['PROCESSO', 'EMENTA', 'ROTULO_MANUAL', 'EMENTA_NORM']

df = pd.read_csv('../data/ementas_pre-processadas.csv', header=0, sep=',', quotechar='"', usecols=colunas)

In [2]:
df.head()

,PROCESSO,EMENTA,ROTULO_MANUAL,EMENTA_NORM
0,00024255420138240081,<br> APELAÇÃO CÍVEL. AÇÃO DE INDENIZAÇÃO POR...,DAN,apelacao civel acao indenizacao danos morais c...
1,00103872520108240020,<br> APELAÇÃO CÍVEL. AÇÃO DE RESCISÃO CONTRA...,CON,apelacao civel acao rescisao contratual cumula...
2,00103872520108240020,<br> EMBARGOS DE DECLARAÇÃO EM APELAÇÃO CÍVE...,CON,embargos declaracao apelacao civel inexistenci...
3,00067390720078240064,<br> EMBARGOS DE DECLARAÇÃO EM APELAÇÃO CÍVE...,PRE,embargos declaracao apelacao civel acordao ape...
4,00067390720078240064,<br> EMBARGOS DE DECLARAÇÃO EM APELAÇÃO CÍVE...,PRE,embargos declaracao apelacao civel previdencia...


In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

vetor = TfidfVectorizer(ngram_range=(1, 3), max_df=1.0, min_df=3)
x = vetor.fit_transform(df['EMENTA_NORM'].values.astype('U'))

In [24]:
vetor.vocabulary_

{'apelacao': 8920,
 'civel': 20988,
 'acao': 509,
 'indenizacao': 64056,
 'danos': 34051,
 'morais': 80378,
 'competencia': 24072,
 'camara': 17838,
 'especial': 48666,
 'chapeco': 20415,
 'lide': 74413,
 'oriunda': 88525,
 'comarca': 23677,
 'pertencente': 93390,
 'viii': 131180,
 'regiao': 109879,
 'judiciaria': 70930,
 'inteligencia': 68188,
 'resolucao': 112605,
 'ato': 13239,
 'regimental': 109921,
 'redistribuicao': 109316,
 'recurso': 108319,
 'nao': 81503,
 'conhecido': 26576,
 'apelacao civel': 8940,
 'civel acao': 20989,
 'acao indenizacao': 778,
 'indenizacao danos': 64095,
 'danos morais': 34187,
 'competencia camara': 24080,
 'camara especial': 17881,
 'oriunda comarca': 88527,
 'ato regimental': 13340,
 'redistribuicao recurso': 109333,
 'recurso nao': 108751,
 'nao conhecido': 82054,
 'apelacao civel acao': 8941,
 'civel acao indenizacao': 21017,
 'acao indenizacao danos': 782,
 'indenizacao danos morais': 64098,
 'redistribuicao recurso nao': 109334,
 'recurso nao conhe

In [25]:
vetor.get_feature_names()

['abaixo',
 'abaixo daquele',
 'abaixo percentual',
 'abaixo percentual divulgado',
 'abalar',
 'abalar higidez',
 'abalar higidez cartula',
 'abalo',
 'abalo animico',
 'abalo animico configurado',
 'abalo animico contudo',
 'abalo animico evidenciado',
 'abalo animico indenizavel',
 'abalo animico mero',
 'abalo animico nao',
 'abalo animico parte',
 'abalo animico passivel',
 'abalo animico presumido',
 'abalo animico sofrido',
 'abalo configurado',
 'abalo credito',
 'abalo credito dano',
 'abalo fisico',
 'abalo honra',
 'abalo honra reputacao',
 'abalo moral',
 'abalo moral configurado',
 'abalo moral nao',
 'abalo moral presumido',
 'abalo moral regra',
 'abalo nao',
 'abalo presumido',
 'abalo psicologico',
 'abalo psicologico sofrimento',
 'abalo psiquico',
 'abalo reputacao',
 'abalo reputacao bom',
 'abalo sofrido',
 'abalo suportado',
 'abalo suportado outro',
 'abalo suportado vitima',
 'abalo vivenciado',
 'abalo vivenciado contrapartida',
 'abalos',
 'abalroado',
 'abalr

In [11]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer()
X_train_tfidf = fidf_transformer.fit_transform(x)
X_train_tfidf.shape



NameError: name 'fidf_transformer' is not defined